# 外皮情報LV1からLV2へのコンバート

## 1. 外皮簡易法＋用途別床面積
- 当該住戸の外皮の部位の面積等を用いずに簡易熱負荷計算を実施する場合には、部位の面積および長さを、「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）」２．エネルギー消費性能の算定方法　2.2　算定方法　第三章　 暖冷房負荷と外皮性能　	第二節　外皮性能　9.当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法　に示された標準住戸（以下、「簡易外皮入力モデル」）における部位の面積および長さ等より推定する。
- 屋根・天井、外壁、床の構成は、入力値である部位種別の熱貫流率をもとに、部位のU値から壁体構成を決定する方法により決定する。
- 土間床等の外周部の線熱貫流率については、簡易熱負荷計算での扱いに従う
- 開口部の日射熱取得率および外部日除けの取得日射熱補正係数は、それぞれ、季節別に、垂直面日射熱取得率および取得日射熱補正係数の入力値もしくは規定値を使用する。この場合、簡易熱負荷計算においては、開口部の入射角特性および外部日除けは考慮しない。
- その他は2.と同様に求める。

#### 《設計住戸の各部位の長さおよび面積の推定》
$$ \qquad
A_{i,d}= A_{i,s}・\frac{A_{floor,d}}{A_{floor,s}}
\qquad (1)
\\
$$
$$ \qquad
L_{i,d}= L_{i,s}・\frac{A_{floor,d}}{A_{floor,s}}
\qquad (2)
\\
$$

- 入力値：

  - 簡易外皮入力モデル住戸の延床面積$[m ^ 2]$：$A_{floor,s}$
  - 設計住戸の延床面積$[m ^ 2]$：$A_{floor,d}$
  - 簡易外皮入力モデル住戸の部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{i,s}$
  - 簡易外皮入力モデル住戸の部位（土間床等の外周部）𝑖の長さ$[m]$：$L_{i,s}$

- 出力値：

  - 設計住戸の部位（一般部位又は開口部）𝑖の面積の推定値$[m ^ 2]$：$A_{i,d}$
  - 設計住戸の部位（土間床等の外周部）𝑖の長さの推定値$[m]$：$L_{i,d}$

## 2. コンバート【分類：共通】

- 地域区分、主たる居室の床面積(m<sup>2</sup>)、その他の床面積(m<sup>2</sup>)、床面積の合計(m<sup>2</sup>)の情報は維持する。
- 新たに変数「簡易外皮入力フラグ」を作成し、LV1から生成したLV2情報は必ずTrueとする。

### Input

```
data = {
    'Region'                     : # 地域の区分 1 ~ 8 の整数
    'MainOccupantRoomFloorArea'  : # 主たる居室の床面積(m2)
    'OtherOccupantRoomFloorArea' : # その他の居室の床面積(m2)
    'TotalFloorArea'             : # 床面積の合計(m2)
}
```

### Output

```
data = {
    'Region'                      : # 地域の区分 1 ~ 8 の整数
    'FlagSimplifiedEnvelopeInput' : # bool値 簡易入力から生成したデータか否かを表す
    'MainOccupantRoomFloorArea'   : # 主たる居室の床面積(m2)
    'OtherOccupantRoomFloorArea'  : # その他の居室の床面積(m2)
    'TotalFloorArea'              : # 床面積の合計(m2)

}
```

### Function

In [1]:
def convert_common(d):
    return {
        'Region'                      : d['Region'],
        'FlagSimplifiedEnvelopeInput' : True,
        'MainOccupantRoomFloorArea'   : d['MainOccupantRoomFloorArea'],
        'OtherOccupantRoomFloorArea'  : d['OtherOccupantRoomFloorArea'],
        'TotalFloorArea'              : d['TotalFloorArea']
    }

### Example

In [2]:
convert_common({
    'Region' : 6,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0,
})

{'FlagSimplifiedEnvelopeInput': True,
 'MainOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloorArea': 60.0,
 'Region': 6,
 'TotalFloorArea': 120.0}

#### コンバート【分類：壁体構成】
- 部位のU値から壁体構成を決定する

- レベル1：
  - 壁体名称$[-]$：$name$
  - 部位種別$[-]$：$type$
  - 平均熱貫流率$[W/(m^2・K)]$：$U_a$
- レベル2：
  - 壁体名称$[-]$：$name$
  - 部位種別$[-]$：$type$
  - <font color="red">部材名称$[-]$：$matrial$</font>
  - <font color="red">厚さ$[m]$：$d$</font>
  - <font color="red">熱伝達率$[W/(m・K)]$：$λ$</font>
  - <font color="red">容積比熱$[J/(m^3・K)]$：$Cρ$</font>
  - 平均熱貫流率$[W/(m^2・K)]$：$U_a$

関数の指定

In [3]:
def convert_1to2_wall(d1):
    import nbimporter    
    #import U_Value_to_Detail_Specification
    #Layers=U_Value_to_Detail_Specification.convert_1to2_wall(d1)
    from U_Value_to_Detail_Specification import convert_UtoDetailSpecification
    Layers=convert_UtoDetailSpecification(d1)
   
    return {'structure': d1['structure'],
            'name': d1['name'],
            'type': d1['type'],
            'Ua': d1['Ua'],
            'Layers':Layers}

関数の実行

In [4]:
convert_1to2_wall( {
    'structure':1,
    'name': 'Sample Wall',
    'type': 2,
    'Ua': 0.538})

Importing Jupyter notebook from U_Value_to_Detail_Specification.ipynb


{'Layers': {'Layers': [{'Cρ': 13, 'd': 0.074, 'material': 'GW16K', 'λ': 0.045},
   {'Cρ': 830, 'd': 0.0095, 'material': 'GPB', 'λ': 0.22}]},
 'Ua': 0.538,
 'name': 'Sample Wall',
 'structure': 1,
 'type': 2}

#### コンバート【分類：開口部】

- レベル1：
  - 開口部名称$[-]$：$name$
  - 日射熱取得率（夏期）$[-]$：$eta_c$
  - 日射熱取得率（冬期）$[-]$：$eta_h$
  - 熱貫流率$[W/(m^2・K)]$：$U$ 
- レベル2：
  - 開口部名称$[-]$：$name$
  - 日射熱取得率（夏期）$[-]$：$eta_c$
  - 日射熱取得率（冬期）$[-]$：$eta_h$
  - 熱貫流率$[W/(m^2・K)]$：$U$ 

関数の指定

In [5]:
def convert_1to2_window(d1):
    return {'name': d1['name'],
            'etac': d1['etac'],
            'etah': d1['etah'],
            'U': d1['U']}

関数の実行

In [6]:
convert_1to2_window({
    'name': 'Sample Window',
    'etac': 0.738,
    'etah': 0.738,
    'U': 4.65})

{'U': 4.65, 'etac': 0.738, 'etah': 0.738, 'name': 'Sample Window'}

#### コンバート【分類：外部日除け】

- レベル1：
  - 外部日除け名称$[-]$：$name$
  - 取得日射熱補正係数（夏期）$[-]$：$f_c$
  - 取得日射熱補正係数（冬期）$[-]$：$f_h$
- レベル2：
  - 外部日除け名称$[-]$：$name$
  - 取得日射熱補正係数（夏期）$[-]$：$f_c$
  - 取得日射熱補正係数（冬期）$[-]$：$f_h$

関数の指定

In [7]:
def convert_1to2_sunshade(d1):
    return {'name': d1['name'],
            'fc': d1['fc'],
            'fh': d1['fh']}

関数の実行

In [8]:
convert_1to2_sunshade({
    'name': 'Sample Sunshade',
    'fc': 0.700,
    'fh': 0.600,})

{'fc': 0.7, 'fh': 0.6, 'name': 'Sample Sunshade'}

#### コンバート【分類：部位情報】

- 簡易外皮入力モデル住戸の部位の長さおよび面積、設計住戸の延床面積より、設計住戸の各部位の長さおよび面積の推定を行う

- レベル1：
  - 外皮フラグ$[-]$：$FlagPerimeter$
  - 日射フラグ$[-]$：$FlagSolarRadiation$
  - 方位$[-]$：$direction$
  - 非定常フラグ$[-]$：$FlagUnsteady$
  - 壁体・開口部名称$[-]$：$name$
  - 部位面積$[m^2]$：$A$
  - 外部日除け名称$[-]$：$sunshade$
- レベル2：
  - 外皮フラグ$[-]$：$FlagPerimeter$
  - 日射フラグ$[-]$：$FlagSolarRadiation$
  - 方位$[-]$：$direction$
  - 非定常フラグ$[-]$：$FlagUnsteady$
  - 壁体・開口部名称$[-]$：$name$
  - 部位面積$[m^2]$：$A$
  - 外部日除け名称$[-]$：$sunshade$

関数の指定

In [9]:
def convert_1to2_element(d1,d1_common):
    FloorArea=d1_common['TotalFloorArea']
    return {'name': d1['name'],
            'FlagPerimeter': d1['FlagPerimeter'],
            'FlagSolarRadiation': d1['FlagSolarRadiation'],
            'FlagUnsteady': d1['FlagUnsteady'],
            'direction': d1['direction'],
            'A': round(d1['A']*FloorArea/90,2)}

関数の実行

In [10]:
convert_1to2_element({
    'name': 'Sample Element',
    'FlagPerimeter': 1,
    'FlagSolarRadiation': 1,
    'FlagUnsteady': 1,
    'direction': 1,
    'A': 1},{    
    'Region' : 6,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0,})

{'A': 1.33,
 'FlagPerimeter': 1,
 'FlagSolarRadiation': 1,
 'FlagUnsteady': 1,
 'direction': 1,
 'name': 'Sample Element'}

#### コンバート【全分類】

In [11]:
def Convert(d):
    d_common = convert_common(d['Common'])

    #標準住戸における部位の面積及び土間床等の外周部の長さ等（い）床断熱住戸の場合
    #番号,外皮フラグ[-],日射フラグ[-],方位[-],非定常フラグ[-],壁体・開口部名称[-],部位面積[㎡],外部日除け名称[-]
    #ElementNo,FlagPerimeter,FlagSolarRadiation,direction,FlagUnsteady,name,A,Sunshade
    StandardHouseElemntsFI = [ [1,1,1,1,1,'Ceiling',50.85,None],
                               [2,1,1,7,1,'Wall',30.47,None],
                               [3,1,1,9,1,'Wall',22.37,None],
                               [4,1,1,3,1,'Wall',47.92,None],
                               [5,1,1,5,1,'Wall',22.28,None],
                               [6,1,1,7,2,'Door',0,None],
                               [7,1,1,9,2,'Door',1.89,None],
                               [8,1,1,3,2,'Door',1.62,None],
                               [9,1,1,5,2,'Door',0,None],
                               [10,1,1,7,2,'Window',22.69,'Sunshade'],
                               [11,1,1,9,2,'Window',2.38,'Sunshade'],
                               [12,1,1,3,2,'Window',3.63,'Sunshade'],
                               [13,1,1,5,2,'Window',4.37,'Sunshade'],
                               [14,1,2,12,1,'Floor',45.05,None],
                               [15,1,1,7,1,'AbaseSW',0,None],
                               [16,1,1,9,1,'AbaseNW',0.91,None],
                               [17,1,1,3,1,'AbaseNE',0.91,None],
                               [18,1,1,5,1,'AbaseSE',0,None],
                               [19,1,2,12,1,'AbaseIF',1.82,None],
                               [20,1,1,7,1,'AbasedSW',0,None],
                               [21,1,1,9,1,'AbasedNW',0.33,None],
                               [22,1,1,3,1,'AbasedNE',0.25,None],
                               [23,1,1,5,1,'AbasedSE',0,None],
                               [24,1,2,12,1,'AbasedIF',0.57,None],
                               [25,1,1,7,1,'LprmSW',0,None],
                               [26,1,1,9,1,'LprmNW',1.82,None],
                               [27,1,1,3,1,'LprmNE',1.82,None],
                               [28,1,1,5,1,'LprmSE',0,None],
                               [29,1,2,12,1,'LprmIF',3.64,None],
                               [30,1,1,7,1,'LprmdSW',0,None],
                               [31,1,1,9,1,'LprmdNW',1.82,None],
                               [32,1,1,3,1,'LprmdNE',1.37,None],
                               [33,1,1,5,1,'prmdSE',0,None],
                               [34,1,2,12,1,'LprmdIF',3.19,None] ]

    def get_StandardHouseElemntsFI(StandardHouseElemntsFI,i):
        data_elements = {'name': StandardHouseElemntsFI[i][5],
                         'FlagPerimeter': StandardHouseElemntsFI[i][1],
                         'FlagSolarRadiation': StandardHouseElemntsFI[i][2],
                         'FlagUnsteady': StandardHouseElemntsFI[i][4],
                         'direction': StandardHouseElemntsFI[i][3],
                         'A': StandardHouseElemntsFI[i][6],
                         'Sunshade': StandardHouseElemntsFI[i][7]}        
        return data_elements

    dictionary_element=[]
    for i in range(0,len(StandardHouseElemntsFI)):
        # dictionary_element=dictionary_element+[[]]
        temporary=convert_1to2_element(get_StandardHouseElemntsFI(StandardHouseElemntsFI,i), d_common)
        if get_StandardHouseElemntsFI(StandardHouseElemntsFI,i)['name']=='Ceiling':
            temporary['Wall']=convert_1to2_wall( {'structure':1,'name': 'Ceiling','type': 1,'Ua': 0.24})
        elif get_StandardHouseElemntsFI(StandardHouseElemntsFI,i)['name']=='Wall':
            temporary['Wall']=convert_1to2_wall( {'structure':1,'name': 'Wall','type': 2,'Ua': 0.53})
        elif get_StandardHouseElemntsFI(StandardHouseElemntsFI,i)['name']=='Floor':
            temporary['Wall']=convert_1to2_wall( {'structure':1,'name': 'Floor','type': 3,'Ua': 0.53})
        elif get_StandardHouseElemntsFI(StandardHouseElemntsFI,i)['name']=='Window':
            temporary['Window']=convert_1to2_window({'name': 'Window','etac': 0.51,'etah': 0.51,'U': 3.49})
            temporary['Sunshade']=convert_1to2_sunshade({'name': 'Sunshade','fc': 0.700,'fh': 0.600,})
        elif get_StandardHouseElemntsFI(StandardHouseElemntsFI,i)['name']=='Door':
            temporary['Door']=convert_1to2_window(
                {'name': 'Door','etac':round(2.33*0.034,3),'etah':round(2.33*0.034,3),'U': 2.33})
        # dictionary_element[i].append(temporary)
        dictionary_element.append(temporary)

    return {'common':d_common,'element':dictionary_element}

In [12]:
d = {
    'Common' : {
        'Region'                     : 6,
        'MainOccupantRoomFloorArea'  : 30.0,
        'OtherOccupantRoomFloorArea' : 60.0,
        'TotalFloorArea'             : 120.0
    }
}

Convert(d)

{'common': {'FlagSimplifiedEnvelopeInput': True,
  'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'Region': 6,
  'TotalFloorArea': 120.0},
 'element': [{'A': 67.8,
   'FlagPerimeter': 1,
   'FlagSolarRadiation': 1,
   'FlagUnsteady': 1,
   'Wall': {'Layers': {'Layers': [{'Cρ': 13,
       'd': 0.179,
       'material': 'GW16K',
       'λ': 0.045},
      {'Cρ': 830, 'd': 0.0095, 'material': 'GPB', 'λ': 0.22}]},
    'Ua': 0.24,
    'name': 'Ceiling',
    'structure': 1,
    'type': 1},
   'direction': 1,
   'name': 'Ceiling'},
  {'A': 40.63,
   'FlagPerimeter': 1,
   'FlagSolarRadiation': 1,
   'FlagUnsteady': 1,
   'Wall': {'Layers': {'Layers': [{'Cρ': 13,
       'd': 0.076,
       'material': 'GW16K',
       'λ': 0.045},
      {'Cρ': 830, 'd': 0.0095, 'material': 'GPB', 'λ': 0.22}]},
    'Ua': 0.53,
    'name': 'Wall',
    'structure': 1,
    'type': 2},
   'direction': 7,
   'name': 'Wall'},
  {'A': 29.83,
   'FlagPerimeter': 1,
   'FlagSolarRadiation': 1,
 